In [1]:
%matplotlib inline
import os
import sys
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
##from sklearn.cross_validation import KFold
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, f1_score

def progress(i, end_val, bar_length=50):
    '''
    Print a progress bar of the form: Percent: [#####      ]
    i is the current progress value expected in a range [0..end_val]
    bar_length is the width of the progress bar on the screen.
    '''
    percent = float(i) / end_val
    hashes = '#' * int(round(percent * bar_length))
    spaces = ' ' * (bar_length - len(hashes))
    sys.stdout.write("\rPercent: [{0}] {1}%".format(hashes + spaces, int(round(percent * 100))))
    sys.stdout.flush()

NEWLINE = '\n'


In [2]:


HAM = 'ham'
SPAM = 'spam'


SOURCES = [
    ('spam/spam',        SPAM),
    ('spam/easy_ham',    HAM),
    ('spam/hard_ham',    HAM),
    ('spam/beck-s',      HAM),
    ('spam/farmer-d',    HAM),
    ('spam/kaminski-v',  HAM),
    ('spam/kitchen-l',   HAM),
    ('spam/lokay-m',     HAM),
    ('spam/williams-w3', HAM),
    ('spam/BG',          SPAM),
    ('spam/GP',          SPAM),
    ('spam/SH',          SPAM)
]

SKIP_FILES = {'cmds'}


def read_files(path):
    '''
    Generator of pairs (filename, filecontent)
    for all files below path whose name is not in SKIP_FILES.
    The content of the file is of the form:
        header....
        <emptyline>
        body...
    This skips the headers and returns body only.
    '''
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
        for file_name in file_names:
            if file_name not in SKIP_FILES:
                file_path = os.path.join(root, file_name)
                if os.path.isfile(file_path):
                    past_header, lines = False, []
                    f = open(file_path, encoding="latin-1")
                    for line in f:
                        if past_header:
                            lines.append(line)
                        elif line == NEWLINE:
                            past_header = True
                    f.close()
                    content = NEWLINE.join(lines)
                    yield file_path, content


def build_data_frame(l, path, classification):
    rows = []
    index = []
    for i, (file_name, text) in enumerate(read_files(path)):
        if ((i+l) % 100 == 0):
            progress(i+l, 58910, 50)##Para ir mostrando el porcentaje de carga de datos
        rows.append({'text': text, 'class': classification})
        index.append(file_name)
   
    data_frame = DataFrame(rows, index=index)
    return data_frame, len(rows)

def load_data():
    data = DataFrame({'text': [], 'class': []})
    l = 0
    for path, classification in SOURCES:
        data_frame, nrows = build_data_frame(l, path, classification)
        data = data.append(data_frame,sort=False)
        l += nrows
    data = data.reindex(numpy.random.permutation(data.index))
    return data

In [3]:
# Cargamos los datos de la carpeta spam, donde se tienen los dataset de los correos
data=load_data()

Percent: [##################################################] 100%

In [4]:
len(data) ##mostramos el largo de caracteristicas obtenidas

58910

In [10]:
data.head()#se muestran algunos datos ya con su texto y su etiqueta y su index como su direccion de archivo

,text,class
spam/kitchen-l/_americas_culture/19,The Management Team follow-up meeting for the ...,ham
spam/GP/part11/msg3330.eml,<html>\n\n<head>\n\n<title>You got it!</title>...,spam
spam/lokay-m/corporate/613,Payroll deductions and cost center billings fo...,ham
spam/GP/part3/msg5709.eml,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
spam/BG/2005/02/1109546715.6669_486.txt,------775aa55dfc-glay.org\n\nContent-Type: tex...,spam


In [19]:
##Antes de entrenar los algoritmos, necesitamos extraer las caracteristicas de los emails, con el fin de reducir la cantidad 
#De datos que se procesaran en un set de atributos del que pueda aprender mas, para eso se usa COuntVectorizer
#de sklearn y magicamente hace operaciones para clasificar y entrenar con feature_extraction
# por otra parte  fit_transform realiza las tareas de aprender del vocabulario y extraer las caraceteristicas del 
#vocabulario
import numpy
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(data['text'].values)
targets = data['class'].values


In [18]:
#Se usara para este caso el clasificador de bayes y la definicion es la siguiente:
#En términos simples, un clasificador de Bayes ingenuo asume que la presencia o ausencia de una característica 
#particular no está relacionada con la presencia o ausencia de cualquier otra característica, dada la clase variable.
#Por ejemplo, una fruta puede ser considerada como una manzana si es roja, redonda y de alrededor de 7 cm de diámetro
# Un clasificador de Bayes ingenuo considera que cada una de estas características contribuye de manera 
#independiente a la probabilidad de que esta fruta sea una manzana, independientemente de la presencia o 
#ausencia de las otras características.

AttributeError: 'CountVectorizer' object has no attribute 'idf_'

In [21]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)#se entrena el modelo para hacer predicciones con bayes

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
##Ejemplo propuesto por la fuente
examples = ['Free Viagra call today!', "I'm going to attend the Linux users group tomorrow."]
example_counts = count_vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions # [1, 0]


array(['spam', 'ham'], dtype='<U4')

In [51]:
#EJemplo propuesto por nosotros Alfredo y Armando jsjsj, parece que la palabra #viagra# y #free# sale en la mayoria
#de los ejemplos de spam, se pueden proponer otros ejemplos para probar como el clasificador de peliculas que se vio
# en clase, por lo que si se omiten este tipo de palabras probablemente dira que el mensaje no es spam.
examples = ['Join to us, and get our free trial right now', "Hello, nice to meet you."]
example_counts = count_vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions # [1, 0]

array(['spam', 'ham'], dtype='<U4')

In [53]:
from sklearn.pipeline import Pipeline
#creamos un pipeline para poder ejecutar los clasificadores antes mencionados
pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('classifier',  MultinomialNB()) ])

pipeline.fit(data['text'].values, data['class'].values)
pipeline.predict(examples) # ['spam', 'ham'] esperados con el ultimo ejemplo propuesto...


array(['spam', 'ham'], dtype='<U4')

In [54]:
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, f1_score

k_fold = KFold( n_splits=6)
scores = []
confusion = numpy.array([[0, 0], [0, 0]])
for train_indices, test_indices in k_fold.split(counts):
    train_text = data.iloc[train_indices]['text'].values
    train_y = data.iloc[train_indices]['class'].values

    test_text = data.iloc[test_indices]['text'].values
    test_y = data.iloc[test_indices]['class'].values

    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=SPAM)
    scores.append(score)

    #obtenemos los resultados de usar la validacion cruzada
print('Total emails classified:', len(data))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

Total emails classified: 58910
Score: 0.9364936082702466
Confusion matrix:
[[23400   139]
 [ 4102 31269]]
